In [1]:
import cv2
import numpy as np
import json
from PIL import Image

In [2]:
#load json file as exported  from Labelbox.

path = '../Downloads/export-2019-06-23T21_23_14.787Z.json'

with open(path) as handle:
    data = json.load(handle)

In [3]:
# Iterate through the cell object dictionary which contains lists of coordinates that will later be used to draw polygons
list_coord= []

# data[0] in the for loop iterates through coordinates of the first image. data[1], for example, iterates through coordinates
# belonging to the second image
for cell in data[0]['Label']['Cell']:
    points = []
    for point in cell['geometry']:
        points.append([point['x'],point['y']])
    list_coord.append(points)

In [4]:
# Convert each list of coordinates corresponding to one polygon into a list of arrays
list_array = []

for cell in list_coord:
    array = np.array(cell)
    list_array.append(array)

In [52]:
# Extract morphological data of each polygon
# Structure: [[area, perimeter, convexity, fit cirlce, fit rectangle,
# aspect_ratio, solidity, extent, equivalent diameter, mean color intensity], [....], [...],.....]
morph_data = []

for polygon_array in list_array:
    l = []
    
    area1 = cv2.contourArea(polygon_array)
    l.append(area)
    
    perimeter = cv2.arcLength(polygon_array,True)
    l.append(perimeter)
    
    k = cv2.isContourConvex(polygon_array)
    l.append(k)
    
    (x,y),radius = cv2.minEnclosingCircle(polygon_array)
    center = (int(x),int(y))
    radius = int(radius)
    tup = [center, radius]
    l.append(tup)
    
    rect = cv2.minAreaRect(polygon_array)
    l.append(rect)
    
    x,y,w,h = cv2.boundingRect(polygon_array)
    aspect_ratio = float(w)/h
    l.append(aspect_ratio)
    
    area2 = cv2.contourArea(polygon_array)
    hull = cv2.convexHull(polygon_array)
    hull_area = cv2.contourArea(hull)
    solidity = float(area)/hull_area
    l.append(solidity)
    
    #area3 = cv2.contourArea(polygon_array)
    #x,y,w,h = cv2.boundingRect(polygon_array)
    #rect_area = w*h
    #extent = float(area)/rect_area
    #l.append(extent)
    
    area = cv2.contourArea(polygon_array)
    equi_diameter = np.sqrt(4*area/np.pi)
    l.append(equi_diameter)
    
    
    #mean_val = cv2.mean(im, polygon_array)
    #l.append(mean_val)
    
    
    morph_data.append(l)
    
    #ellipse = cv2.fitEllipse(polygon_array)
    #l.append(ellipse)
    
    
    
# Run the following lines if you wish to display polygon(s).

#blank_image = np.zeros(shape=[1024, 1366, 3], dtype=np.uint8)    
#cv2.polylines(blank_image, list_array[0], 2, (255,255,255))
#winname = 'Polygon'
#cv2.namedWindow(winname)
#cv2.imshow('winname', blank_image)
#cv2.waitKey()
#cv2.destroyWindow(winname)

# Run the following line if you wish to save the displayed image on your computer as a png
#cv2.imwrite('blank_image04.png',blank_image)

ZeroDivisionError: float division by zero

In [ ]:
# Reads a csv file that will be used to store morphological data
with open('Morph_test.csv', 'r') as in:
    reader = csv.reader(in)
    # read file row by row
    for row in reader:
        print (row)
in.close()

In [ ]:
# Writes previously extracted morphological data to the selected csv file
with open('Morph_test.csv', 'w', newline='') as out:
    writer = csv.writer(out)
    #each row will correspond to all morphological data that describes a single cell
    for cell_data in morph_data:
        writer.writerow(cell_data)


out.close()

In [ ]:
# Draw labels on a blank background

blank_image = np.zeros(shape=[1024, 1366, 3], dtype=np.uint8)

# list_array[0] will only include coordinates of polygons that belong to the first image. list_array[1], for instance,
# will draw labels for the second image.
cv2.polylines(blank_image, list_array, 2, (255,255,255))


winname = 'labels'
cv2.namedWindow(winname)
cv2.imshow('winname', blank_image)
cv2.waitKey()
cv2.destroyAllWindows(winname)

In [ ]:
#Saves generated labels to your computer
cv2.imwrite('img_01.png',blank_image)